In [ ]:
import  numpy as np
import pandas as pd
import importlib
import simuls, utils 
from pycaret.regression import *
importlib.reload(simuls)
importlib.reload(utils)
#ExampleThree = simuls.ExampleThree
from simuls import *
from utils import *

In [ ]:
data = ExampleOne(n = 60, sig = 1, with_test = True).data
data.to_df()
#data.train.to_df(); data.test.to_df()
cols = [col for col in data.df.columns if "x" in col]

In [ ]:
fast_models = ["lr", "br", "lar", "llar", "svm",  "en", "gbr", "omp", "kr"]
slow_models = ["et", "rf", "xgboost", "mlp", "tr", ] 

In [ ]:
res_df = pd.DataFrame({"name":my_models}).set_index("name")
res_df[["model", "mse"]] = None

In [ ]:
tracker = []
for model in fast_models:
    scores = []
    tracker += model
    print("Evaluating "+ model)
    start = time.time(); n_iter = 100
    for i in tqdm(range(n_iter)):
        data = ExampleOne(n = 60, sig=1, with_test = True, seed = i + 1).data
        data.train.to_df(); data.test.to_df()
        out = setup(data = data.train.df, test_data= None, target = "y", silent = True, remove_multicollinearity= True, 
            verbose = False, preprocess = False)
        tuned = tune_model(create_model(model, cross_validation = False, verbose = False), optimize = "MSE", 
            choose_better = True, verbose = False)
        preds = tuned.predict(data.test.df[cols]) 
        scores += [mse(preds, data.test.df["y"])]
    end = time.time() - start 
    name = get_name(tuned)
    res_df.loc[model, ["model", "mse", "time"]] = [name, np.array(scores).mean(), end / n_iter]
    print(res_df.loc[tracker,:])
    

In [ ]:
res_df